In [7]:
#https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/#1introduction
import nltk
nltk.download('stopwords')

import numpy as np
import json
import glob

#Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

#spacy
import spacy
from nltk.corpus import stopwords

#vis
import pyLDAvis
import pyLDAvis.gensim

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

[nltk_data] Downloading package stopwords to C:\Users\Oshadha
[nltk_data]     Abeyrathne\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [8]:
def load_data(file):
    with open (file, "r", encoding="utf-8") as f:
        data = json.load(f) 
    return (data)

def write_data(file, data):
    with open (file, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4)

In [9]:
stopwords = stopwords.words("english")

In [10]:
print(stopwords)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [25]:
data = load_data("data.json")["introduction"]

print (data[0:100])

Keeping up with newest developments and innovations in the continually growing arena of academic stu


In [27]:
def lemmatization(texts, allowed_postags=["NOUN", "ADJ", "VERB", "ADV"]):
    nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
    texts_out = []
    
    doc = nlp(texts)
    new_text = []
    for token in doc:
        if token.pos_ in allowed_postags:
            new_text.append(token.lemma_)
    final = " ".join(new_text)
    texts_out.append(final)
    return (texts_out)


lemmatized_texts = lemmatization(data)
print (lemmatized_texts[0:100])

['keep new development innovation continually grow arena academic study hard challenge crucial need effective categorization classification technique increase research paper publication variety field researcher scholar academic institution always look novel way organize retrieve research paper make more accessible relevant specific interest project introduce novel approach address challenge topic modeling main purpose extract research paper meaningful topic topic modeling categorize topic predefine research domain theme use text classification use short text classification model end result project system significantly enhance efficiency effectiveness research paper management discovery proposal outline key component project include problem domain compare exist work aim research question propose solution objective scope scope requirement tentative plan risk']


In [33]:
def gen_words(texts):
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text, deacc=True)
        final.append(new)
    return (final)

data_words = gen_words(lemmatized_texts)

print (data_words[0][0:10])

['keep', 'new', 'development', 'innovation', 'continually', 'grow', 'arena', 'academic', 'study', 'hard']


In [35]:
id2word = corpora.Dictionary(data_words)

corpus = []
for text in data_words:
    new = id2word.doc2bow(text)
    corpus.append(new)

print (corpus[0][0:20])

word = id2word[[0][:1][0]]
print (word)

[(0, 2), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 2), (10, 3), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 2), (18, 1), (19, 1)]
academic


In [40]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=30,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha="auto")

In [41]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds="mmds", R=30)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
28     0.024153  0.000214       1        1  99.450144
12    -0.000487  0.000026       2        1   0.026108
9     -0.000604  0.000012       3        1   0.026063
10    -0.000646 -0.000018       4        1   0.026051
19    -0.000816 -0.000012       5        1   0.025855
26    -0.000812 -0.000011       6        1   0.025848
27    -0.000859 -0.000009       7        1   0.025495
24    -0.000861 -0.000008       8        1   0.025492
0     -0.000867 -0.000008       9        1   0.016770
17    -0.000867 -0.000010      10        1   0.016770
25    -0.000867 -0.000010      11        1   0.016770
23    -0.000867 -0.000008      12        1   0.016770
22    -0.000867 -0.000007      13        1   0.016770
21    -0.000867 -0.000008      14        1   0.016770
20    -0.000867 -0.000007      15        1   0.016770
18    -0.000867 -0.000008      16        1   0.016770
15    -0.000867 -0.000009      17        1   0.016770
16    -0.000867 -0.000008      18        1   0.016770
1     -0.000867 -0.000008      19        1   0.016770
14    -0.000867 -0.000010      20        1   0.016770
13    -0.000867 -0.000010      21        1   0.016770
11    -0.000867 -0.000007      22        1   0.016770
8     -0.000867 -0.000007      23        1   0.016770
7     -0.000867 -0.000009      24        1   0.016770
6     -0.000867 -0.000011      25        1   0.016770
5     -0.000867 -0.000010      26        1   0.016770
4     -0.000867 -0.000010      27        1   0.016770
3     -0.000867 -0.000011      28        1   0.016770
2     -0.000867 -0.000008      29        1   0.016770
29    -0.000867 -0.000010      30        1   0.016770, topic_info=              Term      Freq     Total Category  logprob  loglift
62        research  5.000000  5.000000  Default  30.0000  30.0000
50           paper  3.000000  3.000000  Default  29.0000  29.0000
79           topic  3.000000  3.000000  Default  28.0000  28.0000
54         project  2.000000  2.000000  Default  27.0000  27.0000
10  classification  2.000000  2.000000  Default  26.0000  26.0000
..             ...       ...       ...      ...      ...      ...
23           exist  0.000214  1.008805  Topic30  -4.4308   0.2332
24         extract  0.000214  1.008678  Topic30  -4.4308   0.2334
25           field  0.000214  1.008935  Topic30  -4.4308   0.2331
26            grow  0.000214  1.008563  Topic30  -4.4308   0.2335
29        increase  0.000214  1.008919  Topic30  -4.4308   0.2331

[1526 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
0         1  1.011591    academic
1         1  0.991807  accessible
2         1  0.991441     address
3         1  0.990924         aim
4         1  0.991583      always
...     ...       ...         ...
79        1  1.021500       topic
80        1  1.010648         use
81        1  0.991728     variety
82        1  0.991624         way
83        1  0.991888        work

[84 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[29, 13, 10, 11, 20, 27, 28, 25, 1, 18, 26, 24, 23, 22, 21, 19, 16, 17, 2, 15, 14, 12, 9, 8, 7, 6, 5, 4, 3, 30])